In [1]:
#Color printing
from termcolor import colored

#General data operations library
import math
import numpy as np

#The tensorflow library
import tensorflow  as tf

#Plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt

#Increase plots font size
params = {'legend.fontsize': 'xx-large',
          'figure.figsize': (10, 7),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

#append path with python modules
import importlib
import sys
sys.path.append("/Users/jakubkusmierski/Desktop/Uczenie_Maszynowe_2/modules")

#Private functions
import plotting_functions as plf
importlib.reload(plf)

import quadratic_equation as qef
importlib.reload(qef);

In [2]:
#Input data preparation
##BEGIN_SOLUTION
x = np.random.default_rng().uniform(-1,1,(150000,3))
y = qef.quadraticEqSolution(x) 

#Select equations with solution
mask = y[:,0]!=None
x = x[mask].astype(float)
y= y[mask].astype(float)

#Sort solutions for each equation
y = np.sort(y, axis=1)

#Training data part
x_val = x[:10000,:]
y_val = y[:10000,:]

#Validation data
x_train = x[10000:,:]
y_train = y[10000:,:]

print(colored("Training data shapes: ","blue"), x_train.shape, y_train.shape)
print(colored("Validation data shapes: ","blue"), x_val.shape, y_val.shape)
#END_SOLUTION
pass

Training data shapes:  (83824, 3) (83824, 2)
Validation data shapes:  (10000, 3) (10000, 2)


/Users/jakubkusmierski/Desktop/Uczenie_Maszynowe_2/modules/quadratic_equation.py:21: RuntimeWarning: invalid value encountered in sqrt
  result = np.where(delta>0, np.sqrt(delta), 0.0)


In [ ]:
%%time
#model definition and training
#BEGIN_SOLUTION
inputShape = (3,)

#nNeurons = np.full((16),64) #1500 epochs x=85% nStepsPerEpoch*15
#nNeurons = np.full((16),128) #1500 epochs x=87% nStepsPerEpoch*15
nNeurons = np.full((16),256) #1500 epoch x=90% nStepsPerEpoch*20

nOutputNeurons = 2

def getModel(inputShape, nNeurons, hiddenActivation="relu", outputActivation="linear", nOutputNeurons=1):
   
    inputs = tf.keras.Input(shape=inputShape, name="features")
    x = inputs
    
    for iLayer, n in enumerate(nNeurons):
        x = tf.keras.layers.Dense(n, activation=hiddenActivation,
                                  kernel_initializer='he_normal',
                                  bias_initializer=tf.keras.initializers.RandomUniform(minval=-1, maxval=1),
                                  name="layer_"+str(iLayer))(x)
                
    outputs = tf.keras.layers.Dense(nOutputNeurons, activation=outputActivation, name = "output")(x)   
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="DNN")
    return model

model = getModel(inputShape=inputShape, nOutputNeurons=nOutputNeurons, 
                 nNeurons=nNeurons, 
                 hiddenActivation="tanh", #it was found by one of students that tanh works much beeter than relu or sigmoid
                 outputActivation="linear")                 

#configure learning process
initial_learning_rate = 1E-3
batchSize = 128
nEpochs = 30

nStepsPerEpoch = len(x_train)/batchSize
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,
                decay_steps=nStepsPerEpoch*20,
                decay_rate=0.95,
                staircase=False)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)
              
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
              loss = 'mape'
             )

#run training
history = model.fit(x=x_train, y=y_train, 
                    epochs=nEpochs, 
                    batch_size = batchSize,
                    validation_data=(x_val, y_val),
                    #callbacks=[callback], #early stopping terminates trainig too early, as loss function is bumpy
                    verbose=0)
plf.plotTrainHistory(history)

#performance on training dataset
print(colored("Training dataset.","blue"))
y_pred = model.predict(x_train)
qef.plotSqEqSolutions(x_train, y_train, y_pred)

#performance on validation dataset
print(colored("Validation dataset.","blue"))
y_pred = model.predict(x_val)
qef.plotSqEqSolutions(x_val, y_val, y_pred)
#END_SOLUTION
pass